In [1]:
import numpy as np

### Instantiate the prediction task

In [2]:
from jenga.tasks.income import IncomeEstimationTask

task = IncomeEstimationTask(seed=42)

### Task details

In this task, we try to predict the income level of a person (more or less than 50K dollars per years) from demographic and work-related data. It is often used as a proxy to study automated decision making for loan applications.

The original data is available in the UCI machine learning repository at https://archive.ics.uci.edu/ml/datasets/adult

In [3]:
task.train_data

,workclass,occupation,marital_status,education,hours_per_week,age
3874,Private,Exec-managerial,Never-married,Some-college,40,22
13112,Private,Exec-managerial,Married-civ-spouse,Bachelors,45,60
17805,Local-gov,Prof-specialty,Divorced,Masters,25,60
17675,Self-emp-not-inc,Sales,Married-civ-spouse,Some-college,40,68
6004,Private,Sales,Married-civ-spouse,Assoc-acdm,48,52
...,...,...,...,...,...,...
19219,Private,Sales,Married-civ-spouse,Assoc-acdm,45,37
27436,Private,Tech-support,Never-married,Bachelors,45,38
20172,Private,Other-service,Married-civ-spouse,7th-8th,40,63
9799,Private,Exec-managerial,Married-civ-spouse,Some-college,55,52


### Train the provided baseline model

Jenga allows us to easily train and evaluate a logistic regression model for this task. Have a look at https://github.com/schelterlabs/jenga/blob/master/jenga/tasks/income.py if you want to know the details.


In [4]:
model = task.fit_baseline_model(task.train_data,task.train_labels)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    3.1s finished


In [5]:
y_pred = model.predict_proba(task.test_data)

f"The ROC AUC score on the test data is {task.score_on_test_data(y_pred)}"

'The ROC AUC score on the test data is 0.8787212685178679'

### Let's have a look at the test data

In [6]:
task.test_data

,workclass,occupation,marital_status,education,hours_per_week,age
22273,Federal-gov,Prof-specialty,Widowed,Masters,40,61
20658,Private,Other-service,Separated,9th,40,36
2898,Local-gov,Prof-specialty,Never-married,Masters,40,29
26845,Private,Tech-support,Married-civ-spouse,Assoc-voc,40,29
28814,Local-gov,Exec-managerial,Married-civ-spouse,Bachelors,42,38
...,...,...,...,...,...,...
904,Private,Sales,Never-married,HS-grad,45,35
27829,Local-gov,Prof-specialty,Divorced,Bachelors,40,36
11216,Private,Prof-specialty,Married-civ-spouse,Masters,20,66
694,State-gov,Tech-support,Never-married,Bachelors,20,23


### Corruptions

Jenga provides a set of predefined data corruptions that we can use to simulate errors in the test data. We will simulate 'implicit missing values' in a column and see how this impacts the prediction quality.

Jenga supports much more predefined corruptions, have a look at https://github.com/schelterlabs/jenga/tree/master/jenga/corruptions.


In [7]:
from jenga.corruptions.generic import MissingValues

marital_status_corruption = MissingValues(column='marital_status', fraction=0.99, na_value='BROKEN')


In [8]:
corrupted_test_data = marital_status_corruption.transform(task.test_data)
corrupted_test_data

,workclass,occupation,marital_status,education,hours_per_week,age
22273,Federal-gov,Prof-specialty,BROKEN,Masters,40,61
20658,Private,Other-service,BROKEN,9th,40,36
2898,Local-gov,Prof-specialty,BROKEN,Masters,40,29
26845,Private,Tech-support,BROKEN,Assoc-voc,40,29
28814,Local-gov,Exec-managerial,BROKEN,Bachelors,42,38
...,...,...,...,...,...,...
904,Private,Sales,BROKEN,HS-grad,45,35
27829,Local-gov,Prof-specialty,BROKEN,Bachelors,40,36
11216,Private,Prof-specialty,BROKEN,Masters,20,66
694,State-gov,Tech-support,BROKEN,Bachelors,20,23


In [9]:
y_pred = model.predict_proba(corrupted_test_data)

f"The ROC AUC score on the corrupted test data is {task.score_on_test_data(y_pred)}"

'The ROC AUC score on the corrupted test data is 0.8110489186579961'

### Jenga's evaluators

Jenga provides a set of evaluators which allow us to automate the evaluation of the impact of given data corruptions.

In [10]:
from jenga.evaluation.corruption_impact import CorruptionImpactEvaluator

evaluator = CorruptionImpactEvaluator(task)

In [11]:
corruptions = [
    MissingValues(column='marital_status', fraction=0.99, na_value='BROKEN'),
    MissingValues(column='age', fraction=0.05, na_value=-999),
]

### Run the evaluation of the corruptions with 10 repetitions 

In [12]:
num_repetitions = 10
results = evaluator.evaluate(model, num_repetitions, *corruptions)

0/20 (0.01639400000000002)
10/20 (0.18642299999999956)


### Investigate the impact on the predictive performance of the model

In [13]:
for validation_result in results:
    
    print(validation_result.corruption)
    print(f"""
     Score (AUC) on 
      clean data:     {validation_result.baseline_score}
      corrupted data: {np.mean(validation_result.corrupted_scores)}
     """)    
    print("\n")

MissingValues: {'column': 'marital_status', 'fraction': 0.99, 'sampling': 'MCAR', 'na_value': 'BROKEN'}

     Score (AUC) on 
      clean data:     0.8787212685178679
      corrupted data: 0.8107457304490607
     


MissingValues: {'column': 'age', 'fraction': 0.05, 'sampling': 'MCAR', 'na_value': -999}

     Score (AUC) on 
      clean data:     0.8787212685178679
      corrupted data: 0.8416967740592416
     


